<a href="https://colab.research.google.com/github/aizazaziz/ML_Projects/blob/main/cats_dogs_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image as keras_image
import os


In [ ]:


TARGET_SIZE = (224, 224)

CLASS_NAMES = {0: 'cat', 1: 'dog'}
MODEL_PATH = "cat_dog_classifier.keras"

if not os.path.exists(MODEL_PATH):
    st.error(f"Error: Model file '{MODEL_PATH}' not found. Please ensure the model is trained and saved.")
    st.stop()



@st.cache_resource
def load_trained_model():
    """Loads the Keras model once."""
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        return model
    except Exception as e:
        st.error(f"Failed to load model: {e}")
        st.stop()


model = load_trained_model()


# --- 3. Prediction Function ---
def predict_image(uploaded_image, model):
    """Processes the image and makes a prediction."""
    # Resize and convert image to array
    img = uploaded_image.resize(TARGET_SIZE)
    img_array = keras_image.img_to_array(img)

    # Prepare batch for model (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)

    # Rescale/Normalize (must match the ImageDataGenerator's rescale=1./255)
    img_array /= 255.0

    # Make prediction
    # model.predict returns an array like [[0.987]] or [[0.012]]
    prediction = model.predict(img_array)[0][0]

    # Interpret binary prediction
    if prediction >= 0.5:
        label = CLASS_NAMES[1]
        confidence = prediction
    else:
        label = CLASS_NAMES[0]
        confidence = 1 - prediction

    return label, confidence


# --- 4. Streamlit UI ---
st.title("🐾 Cat vs. Dog CNN Classifier")
st.markdown("Upload an image to see if the model predicts a Cat or a Dog.")

# File Uploader component
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)
    st.write("")

    if st.button('Classify Image'):
        with st.spinner('Analyzing the image...'):
            label, confidence = predict_image(image, model)

            # Display results
            st.success(f"**Prediction:** This is a **{label.upper()}**!")
            st.info(f"Confidence: **{confidence:.2f}** (Probability)")